In [1]:
#Dans le Machine Learning, les règles de bifurcation se feront à partir de critères statistiques sur les variables.

#Par exemple 

In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [17]:
#si [hauteur_m > 10 & libelle_francais = 'Platane' ]
#Alors stade de développement = Mature
#Sinon [autre règle]

In [18]:
#chargeons les données

In [19]:
filename = 'https://raw.githubusercontent.com/OpenClassrooms-Student-Center/8063076-Initiez-vous-au-Machine-Learning/master/data/paris-arbres-numerical-2023-09-10.csv'

In [20]:
data = pd.read_csv(filename)

In [21]:
data.head()

,domanialite,arrondissement,libelle_francais,genre,espece,circonference_cm,hauteur_m,stade_de_developpement
0,1,8,30,18,36,65.0,8.0,2
1,1,8,2,2,2,25.0,5.0,1
2,3,7,10,10,12,140.0,10.0,3
3,3,7,5,5,14,105.0,10.0,3
4,3,16,4,4,18,263.0,25.0,4


In [22]:
X = data[['domanialite', 'arrondissement', 'libelle_francais', 'genre', 'espece', 'circonference_cm', 'hauteur_m']]

In [23]:
y = data.stade_de_developpement.values

In [24]:
X_train, X_test, y_train, y_test = train_test_split( X, y, train_size=0.8, random_state=808)

In [ ]:
##Limitons la profondeur de l'arbre à 3 et entraînons le modèle :

In [25]:
clf = DecisionTreeClassifier(
        max_depth = 3,
    random_state = 808
)
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, random_state=808)

In [26]:
#Pour le score, on utilise l'AUC

In [28]:
train_auc = roc_auc_score(y_train, clf.predict_proba(X_train), multi_class='ovr')
test_auc = roc_auc_score(y_test, clf.predict_proba(X_test), multi_class='ovr')
print("train",train_auc)
print("test", test_auc)

train 0.8877114343964885
test 0.8882428014767341


In [30]:
#Voici la matrice de confusion 

In [31]:
from sklearn.metrics import confusion_matrix
y_train_hat = clf.predict(X_train)
y_test_hat = clf.predict(X_test)
print(confusion_matrix(y_test, y_test_hat))

[[ 5570  1402   300     2]
 [ 1060  3847  2750     8]
 [  211  1481 13328   466]
 [    4     7   565   877]]


In [ ]:
#ainsi que le rapport de classification qui donne plusieurs scores :

In [33]:
print(classification_report(y_test, y_test_hat))

              precision    recall  f1-score   support

           1       0.81      0.77      0.79      7274
           2       0.57      0.50      0.53      7665
           3       0.79      0.86      0.82     15486
           4       0.65      0.60      0.63      1453

    accuracy                           0.74     31878
   macro avg       0.70      0.68      0.69     31878
weighted avg       0.73      0.74      0.74     31878



In [34]:
#On voit que la  precision  (le ratio de bonne pioche parmi tous les positifs) est faible pour les catégories  Jeune (arbre)Adulte  et  Mature  (respectivement 0,57 et 0,67) et nous observons un recall de 0,5 pour la catégorie  Jeune (arbre)  aussi très faible.

#Donc ce modèle a besoin de tendresse.
#C'est un bon exemple d'un modèle biaisé.

#Notez cependant que le score(test) et le score(train) sont presque égaux :

In [35]:
clf.score(X_train, y_train)

0.7409850207826837

In [36]:
clf.score(X_test, y_test)

0.741012610577828

In [37]:
##Quels remèdes pour minimiser le biais du modèle ?

In [38]:
#Identifiez et compensez l'overfit

In [39]:
#Reprenons maintenant notre arbre de décision, mais cette fois sans limiter sa profondeur.

#Pour cela on fixe la valeur  max_depth = None  .

In [40]:
clf = DecisionTreeClassifier(
    max_depth = None,
    random_state = 808
)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8060104147060668

In [41]:
#Le modèle est en effet meilleur : on passe de 0,74 à 0,81. Mais on remarque que sur le sous-ensemble 
#d'entraînement on a carrément 0,94 !

In [42]:
clf.score(X_train, y_train)
0.935048231511254

0.935048231511254

In [43]:
#Nous sommes bien dans un cas d'overfitting où :

    #le modèle colle aux données d'entraînement (le score(train) est excellent) ;

    #et ne sait pas reproduire la même performance sur les données de test (le score(test) est faible).

In [44]:
#Donc à un moment, entre  maxdepth = 3  et  max_depth = infini  , les score(train) et score(test) ont divergé. Il nous faut trouver un juste milieu pour ce paramètre.

#Faisons croître  max_depth  en enregistrant les scores sur train et test.

#L'AUC est plus parlante que l'accuracy pour cette démonstration :

In [45]:
scores = []
for depth in np.arange(2, 30, 2):
    clf = DecisionTreeClassifier(
        max_depth = depth,
        random_state = 808
    )

    clf.fit(X_train, y_train)

    train_auc = roc_auc_score(y_train, clf.predict_proba(X_train), multi_class='ovr')
    test_auc = roc_auc_score(y_test, clf.predict_proba(X_test), multi_class='ovr')
    scores.append({
        'max_depth': depth,
        'train': train_auc,
        'test': test_auc,
    })

scores = pd.DataFrame(scores)

In [46]:
#En comparant les scores de performance sur les sous-ensembles d'entraînement et de test, on peut distinguer deux cas de sous-performance du modèle :

#le sous-apprentissage, ou biais, aboutit à un score faible sur les 2 sous-ensembles. L'ajout de nouvelles données ou la sélection de paramètres plus efficaces pour le modèle sont des stratégies de remédiation classiques ;

#le sur-apprentissage, ou overfit, se traduit par un écart significatif entre les scores sur les sous-ensembles d'entraînement (élevé) et de test (bien plus faible).

In [ ]:
##Créer un dataset simple à une variable prédictrice avec make_regression.

In [48]:
from sklearn.datasets import make_regression
X, y = make_regression(n_samples=30, n_features=1, noise=40, random_state=200)

In [50]:
#Entraîner une régression linéaire simple y ~ x   avec le modèle Ridge et sans régularisation

In [51]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=0)
model.fit(X, y)
y_pred = model.predict(X)

In [ ]:
##Étape 2 : L'overfit
#Considérons ensuite la régression polynomiale de degré 12 (autrement plus complexe que la simple régression linéaire).

# y=x+x2+...+x12

#Pour créer la matrice des prédicteurs, nous utilisons la transformation PolynomialFeatures :

In [53]:
from sklearn.preprocessing import PolynomialFeatures


In [56]:
#pol = PolynomialFeatures(degree, include_bias = False)
#X_poly = pol.fit_transform(X)
#model = Ridge(alpha=0)
#model.fit(X_poly,  y)

In [57]:
#Étape 3 : La régularisation

In [58]:
#Implémentez la validation croisée

In [59]:
#Il y a de multiples façons de faire de la validation croisée dans scikit-learn.

#Ma méthode préférée pour la sélection des paramètres est GridSearchCV  .
#Elle offre un juste milieu entre automatisation totale et implémentation manuelle complète de la méthode.

In [60]:
from sklearn.model_selection import GridSearchCV

In [61]:
#Définissons l'espace des valeurs possibles du paramètre :

In [62]:
parameters = {'max_depth':np.arange(2, 30, 2)}

In [63]:
#On ne précise pas le paramètre lorsque l'on instancie le modèle :

In [64]:
model = DecisionTreeClassifier(
    random_state = 808
)

In [65]:
#On passe le modèle et le dictionnaire des paramètres à   GridSearchCV  .
#Le paramètre  cv  correspond au nombre de sous-ensembles, de splits de la validation croisée. 
#On choisit le plus souvent une valeur autour de  cv = 5 

In [66]:
clf = GridSearchCV(model, parameters, cv = 5, scoring = 'roc_auc_ovr', verbose = 1)
clf.fit(X, y)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


/home/gabriel/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/gabriel/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/gabriel/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 898, in fit
    super().fit(
  File "/home/gabriel/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 190, in fit
    check_classification_targets(y)
  File "/home/gabriel/anaconda3/lib/python3.8/site-packages/sklearn/utils/multiclass.py", line 183, in check_classification_targets
    raise ValueError("Unknown label type: %r" % y_type)
ValueError: Unknown label type: 'continuous'

  warnings.warn("Estimator fit failed. The score on this train-te

ValueError: Unknown label type: 'continuous'

In [67]:
#L'objet  clf  permet de voir tout de suite…

#    … la meilleure valeur des paramètres 

In [68]:
clf.best_params_

{'max_depth': 2}

In [69]:
# le meilleur score obtenu :

In [70]:
clf.best_score_

nan

In [71]:
#et le meilleur modèle :

In [73]:
clf.best_estimator_


DecisionTreeClassifier(max_depth=2, random_state=808)

In [74]:
#En résumé

In [ ]:
   # La régularisation permet d'éviter que le modèle colle trop aux données d'apprentissage et perde sa capacité d'extrapolation à des données fraîches.

  #  La régularisation prend des formes différentes en fonction des modèles envisagés. On retiendra la profondeur des arbres de décision et les régularisations de type L2 et L1, appelées aussi Ridge et Lasso.

  #  La validation croisée permet de s'affranchir de l'influence des échantillons hors normes en moyennant l'entraînement du modèle sur plusieurs sous-ensembles de test et d'entraînement. 

  #  La fonction  GridSearchCV  permet à la fois d'implémenter une validation croisée et de sélectionner les paramètres optimaux du modèle.